<h1 align=center>Redes Adversariais Geradoras para Sintetizar Novos Dados</h1>

<p align=center><img src=https://advancedinstitute.ai/wp-content/uploads/2020/08/real_fake.png width=500></p>

Anteriomente, focamos em redes neurais recorrentes (RNN) para modelagem de sequências. Aqui, exploraremos as redes generativas de adversários (GANs) e veremos sua aplicação na síntese de novas amostras de dados. As GANs são consideradas o avanço mais importante no aprendizado profundo, permitindo que os computadores gerem novos dados (como novas imagens).

Aqui, abordaremos os seguintes tópicos:
* Apresentando modelos generativos para sintetizar novos dados
* Autoencoders, autoencoders variacionais (VAEs) e seu relacionamento com GANs
* Entendendo os blocos de construção das GANs
* Implementação de um modelo GAN simples para gerar dígitos manuscritos
* Noções básicas sobre convolução transposta e normalização de lote (BatchNorm ou BN)
* Melhorar GANs: GANs convolucionais profundos e GANs usando a distância de Wasserstein

## Apresentando redes adversárias generativas
Vamos primeiro olhar para os fundamentos dos modelos GAN. O objetivo geral de uma GAN é <u>sintetizar novos dados que tenham a mesma distribuição que seu conjunto de dados de treinamento</u>. Portanto, as GANs, em sua forma original, são consideradas na categoria de aprendizado não supervisionado de tarefas de aprendizado de máquina, pois não são necessários dados rotulados. Vale a pena notar, no entanto, que as extensões feitas ao GAN original podem estar em tarefas semi-supervisionadas e supervisionadas.

O conceito geral de GAN foi proposto pela primeira vez em 2014 por Ian Goodfellow e seus colegas como um método para sintetizar novas imagens usando redes neurais profundas (NNs). Embora a arquitetura GAN inicial proposta fosse baseada em camadas totalmente conectadas, semelhantes às arquiteturas *perceptron* multicamadas, e treinadas para gerar dígitos manuscritos de baixa resolução do tipo MNIST, ela serviu mais como uma prova de conceito para demonstrar a viabilidade dessa nova abordagem.

No entanto, desde sua introdução, os autores originais, assim como muitos outros pesquisadores, propuseram inúmeras melhorias e diversas aplicações em diferentes campos da engenharia e da ciência; por exemplo, em visão computacional, GANs são usadas ​​para tradução de imagem para imagem (aprender a mapear uma imagem de entrada para uma imagem de saída), super-resolução de imagem (criar uma imagem de alta resolução a partir de uma versão de baixa resolução), pintura interna de imagem (aprender a reconstruir as partes que faltam de uma imagem) e muitas outras aplicações. Por exemplo, os avanços recentes na pesquisa de GAN levaram a modelos capazes de gerar novas imagens de rosto de alta resolução. Exemplos dessas imagens de alta resolução podem ser encontrados em https://www.thispersondoesnotexist.com/, que mostra imagens de rosto sintéticas geradas por uma GAN.

### Começando com os *autoencoders*
Antes de discutirmos como as GANs funcionam, começaremos com os *autoencoders*, que podem compactar e descompactar dados de treinamento. Embora os codificadores automáticos padrão não possam gerar novos dados, entender sua função ajudará você a navegar pelas GANs na próxima seção.

Autoencoders são compostos por duas redes concatenadas: uma rede codificadora e uma rede decodificadora. A rede codificadora recebe um vetor de recurso de entrada *d*-dimensional associado ao exemplo **x** (ou seja, ***x*** $\small \in R^p$) e o codifica em um vetor *p*-dimensional, `z` (ou seja, ***z*** $\small \in R^p$). Em outras palavras, o papel do codificador é aprender a modelar a função ***z*** = $\small f(x)$ . O vetor codificado, `z`, também é chamado de vetor latente ou representação de característica latente. Normalmente, a dimensionalidade do vetor latente é menor que a dos exemplos de entrada; em outras palavras, $\small p < d$. Assim, podemos dizer que o codificador atua como uma função de compressão de dados. Então, o decodificador descomprime $\small \hat{x}$ do vetor latente de menor dimensão, `z`, onde podemos pensar no decodificador como uma função, $\small \hat{x} = g(z)$. Uma arquitetura simples de *autoencoder* é mostrada na figura a seguir, onde as partes do codificador e do decodificador consistem em apenas uma camada totalmente conectada cada:

![](imagens\autoencoder.PNG)

> ##### A conexão entre autoencoders e redução de dimensionalidade
> Certamente você já aprendeu sobre técnicas de redução de dimensionalidade, como análise de componentes principais (PCA) e análise discriminante linear (LDA). *Autoencoders* também podem ser usados como uma **técnica de redução de dimensionalidade**. De fato, quando não há não linearidade em nenhuma das duas sub-redes (codificador e decodificador), a abordagem do autoencoder é quase idêntica ao PCA.
>
> Nesse caso, se assumirmos que os pesos de um codificador de camada única (sem camada oculta e sem função de ativação não linear) são denotados pela matriz **U**, então os modelos do codificador **z** = $\small U^Tx$. Da mesma forma, um decodificador linear de camada única modela $\small \hat{x} = Uz$. Juntando esses dois componentes, temos $\small \hat{x} = UU^Tx$. Isso é exatamente o que o PCA faz, com a exceção de que o PCA tem uma restrição ortonormal adicional: $\small UU^T = I_{n \times n}$. 

Embora a figura anterior represente um *autoencoder* sem camadas ocultas dentro do codificador e do decodificador, podemos, é claro, adicionar várias camadas ocultas com não linearidades (como em uma NN multicamada) para construir um *autoencoder* profundo que pode aprender funções de reconstrução e compactação de dados mais eficazes. Além disso, observe que o *autoencoder* mencionado nesta seção usa camadas totalmente conectadas. Quando trabalhamos com imagens, no entanto, podemos substituir as camadas totalmente conectadas por camadas convolucionais.

> ##### Outros tipos de autoencoders baseados no tamanho do espaço latente
> Como mencionado anteriormente, a dimensionalidade do espaço latente de um *autoencoder* é tipicamente menor que a dimensionalidade das entradas ($\small p < d$), o que torna os autoencoders adequados para redução de dimensionalidade. Por esse motivo, o vetor latente também é frequentemente chamado de "gargalo", e essa configuração específica de um *autoencoder* também é chamada de `undercomplete` (subcompleto). No entanto, existe uma categoria diferente de *autoencoders*, chamada `overcomplete`, onde a dimensionalidade do vetor latente, `z`, é, de fato, maior que a dimensionalidade dos exemplos de entrada ($\small p > d$).

Ao treinar um *autoencoder* `overcomplete` (supercompleto), há uma solução trivial em que o codificador e o decodificador podem simplesmente aprender a copiar (memorizar) os recursos de entrada para sua camada de saída. Obviamente, esta solução não é muito útil. No entanto, com algumas modificações no procedimento de treinamento, os *autoencoders* supercompletos podem ser usados **​​para redução de ruído**. Nesse caso, durante o treinamento, o ruído aleatório, $\epsilon$, é adicionado aos exemplos de entrada e a rede aprende a reconstruir o exemplo, `x`, do sinal ruidoso, $\small x + \epsilon$. Então, no momento da avaliação, fornecemos os novos exemplos que são naturalmente ruidosos (ou seja, o ruído já está presente de modo que nenhum ruído artificial adicional, $\epsilon$, é adicionado) para remover o ruído existente desses exemplos. Essa arquitetura de *autoencoder* e o método de treinamento específicos são chamados de *autoencoder* de redução de ruído.

### Modelos generativos para sintetizar novos dados

Autoencoders são modelos determinísticos, o que significa que depois que um autoencoder é treinado, dada uma entrada, **x**, ele será capaz de reconstruir a entrada de sua versão compactada em um espaço de menor dimensão. Portanto, ele não pode gerar novos dados além de reconstruir sua entrada por meio da transformação da representação compactada.

Um modelo generativo, por outro lado, pode gerar um novo exemplo, $\small \tilde{x}$ , a partir de um vetor aleatório, **z** (correspondente à representação latente). Uma representação esquemática de um modelo generativo é mostrada na figura a seguir. O vetor aleatório, **z**, vem de uma distribuição simples com características totalmente conhecidas, então podemos facilmente amostrar de tal distribuição. Por exemplo, cada elemento de **z** pode vir da distribuição uniforme no intervalo [–1, 1] (para a qual escrevemos $\small z_i \sim \: Uniform(-1,1)$ ou de uma distribuição normal padrão (nesse caso, escrevemos $\small z_i \sim \: Normal (\mu = 0, \sigma^2=1)$).

<p><img src =imagens\generative_model.PNG></p>


À medida que mudamos nossa atenção de autoencoders para modelos generativos, você deve ter notado que o componente decodificador de um *autoencoder* tem algumas semelhanças com um modelo generativo. Em particular, ambos recebem um vetor latente, **z**, como entrada e retornam uma saída no mesmo espaço que **x**. (Para o *autoencoder*, $\small \hat{x}$ é a reconstrução de uma entrada, **x**, e para o modelo generativo, $\small \tilde{x}$ é uma amostra sintetizada.)

No entanto, a principal diferença entre os dois é que não conhecemos a distribuição de **z** no *autoencoder*, enquanto em um modelo generativo, a distribuição de **z** é totalmente caracterizável. No entanto, é possível generalizar um *autoencoder* em um modelo generativo. Uma abordagem são os **VAEs**. Em um VAE que recebe um exemplo de entrada, **x**, a rede do codificador é modificada de tal forma que computa dois momentos da distribuição do vetor latente: a média, $\small \mu$, e a variância, $\small \sigma^2$. Durante o treinamento de um VAE, a rede é forçada a combinar esses momentos com os de uma distribuição normal padrão (ou seja, **média zero e variância unitária**). Então, depois que o modelo VAE é treinado, o codificador é descartado e podemos usar a rede do decodificador para gerar novos exemplos, $\small \tilde{x}$, alimentando vetores **z** aleatórios da distribuição gaussiana "aprendida". Além dos VAEs, existem outros tipos de modelos generativos, por exemplo, modelos autorregressivos e modelos de fluxo normalizador. No entanto, vamos nos concentrar apenas nos modelos GAN, que estão entre os tipos mais recentes e populares de modelos generativos em deep learning.


> ##### O que é um modelo generativo?
> Observe que os modelos generativos são tradicionalmente definidos como algoritmos que modelam as distribuições de entrada de dados, $\small p(x)$, ou as distribuições conjuntas dos dados de entrada e alvos associados, $\small p(x, y)$. Por definição, esses modelos também são capazes de amostrar de algum recurso, $\small x_i$, condicionado a outro recurso, $\small x_j$, conhecido como **inferência condicional**. No contexto de aprendizado profundo, no entanto, o termo **modelo generativo** é normalmente usado para se referir a modelos que geram dados de aparência realista. Isso significa que podemos amostrar a partir de distribuições de entrada, $\small p(x)$, mas não somos necessariamente capazes de realizar inferência condicional.

### Gerando novas amostras com GANs
Para entender o que as GANs fazem em poucas palavras, vamos primeiro supor que temos uma rede que recebe um vetor aleatório, **z**, amostrado de uma distribuição conhecida e gera uma imagem de saída, **x**. Chamaremos esse **gerador de rede** (G) e usaremos a notação $\small \tilde{x} = G(z)$ para nos referirmos à saída gerada. Suponha que nosso objetivo seja gerar algumas imagens, por exemplo, imagens de rostos, imagens de prédios, imagens de animais ou até mesmo dígitos manuscritos como o MNIST.

Como sempre, inicializaremos essa rede com pesos aleatórios. Portanto, as primeiras imagens de saída, antes que esses pesos sejam ajustados, parecerão ruído branco. Agora, imagine que existe uma função que pode avaliar a qualidade das imagens (vamos chamá-la de **função avaliadora**).

Se tal função existir, podemos usar o feedback dessa função para informar à nossa rede geradora como ajustar seus pesos para melhorar a qualidade das imagens geradas. Dessa forma, podemos treinar o gerador com base no feedback dessa função avaliadora, de modo que o gerador aprenda a melhorar sua saída para produzir imagens de aparência realista.

Enquanto uma função de avaliador, conforme descrito no parágrafo anterior, facilitaria muito a tarefa de geração de imagens, a questão é se existe uma função tão universal para avaliar a qualidade das imagens e, em caso afirmativo, como ela é definida. Obviamente, como humanos, podemos facilmente avaliar a qualidade das imagens de saída quando observamos as saídas da rede; embora não possamos (ainda) retropropagar o resultado do nosso cérebro para a rede. Agora, se nosso cérebro pode avaliar a qualidade das imagens sintetizadas, podemos projetar um modelo NN para fazer a mesma coisa? Na verdade, essa é a ideia geral de uma GAN. Conforme mostrado na figura a seguir, um modelo GAN consiste em uma NN adicional chamado **discriminador** (D), que é um classificador que aprende a detectar uma imagem sintetizada, $\small \tilde{x}$, a partir de uma imagem real, **x**:

<p><img src = imagens\gerador_discriminador.PNG></p>

Em um modelo GAN, as duas redes, geradora e discriminadora, são treinadas juntas. A princípio, após inicializar os pesos do modelo, o gerador cria imagens que não parecem realistas. Da mesma forma, o discriminador faz um trabalho ruim ao distinguir entre imagens reais e imagens sintetizadas pelo gerador. Mas com o tempo (ou seja, por meio de treinamento), ambas as redes se tornam melhores à medida que interagem entre si. De fato, as duas redes fazem um jogo de adversários, onde o gerador aprende a melhorar sua saída para poder enganar o discriminador. Ao mesmo tempo, o discriminador se torna melhor na detecção das imagens sintetizadas.



### Entendendo as funções de perda das redes geradoras e discriminadoras em um modelo GAN
A função objetivo das GANs, conforme descrito no artigo original Generative Adversarial Nets de Goodfellow et al. (https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf), é o seguinte:

$$
\small V\left ( \theta ^{D}, \theta^{(G)} \right ) = E_{x \sim p_{data}(x))}[logD(x)] + E_{z\sim p_z(z))}\left [ log(1-D(G(z))) \right ]
$$

Aqui, $\small V\left ( \theta ^{D}, \theta^{(G)} \right )$ é chamada de *função valor*, que pode ser interpretada como uma recompensa: queremos maximizar seu valor em relação ao discriminador (D), enquanto minimizamos seu valor em relação a o gerador (G), ou seja, min $\small \underset{G}{min} \: \underset{D}{max} \: V(\theta^{(D)}, \theta^{(G)})$.
$D_{(x)}$ é a probabilidade que indica se o exemplo de entrada, **x**, é real ou falso (ou seja, gerado). A expressão $\small E_{x \sim p_{data}(x))}[logD(x)]$ refere-se ao valor esperado da quantidade entre parênteses em relação aos exemplos da distribuição de dados (distribuição dos exemplos reais); $\small E_{z\sim p_z(z))}\left [ log(1-D(G(z))) \right]$ refere-se ao valor esperado da quantidade em relação à distribuição dos vetores de entrada, **z**.

Uma etapa de treinamento de um modelo GAN com tal função de valor requer duas etapas de otimização: (1) maximizar o retorno para o discriminador e (2) minimizar o retorno para o gerador. Uma maneira prática de treinar GANs é alternar entre essas duas etapas de otimização: (1) corrigir (congelar) os parâmetros de uma rede e otimizar os pesos da outra e (2) corrigir a segunda rede e otimizar a primeira. Este processo deve ser repetido a cada iteração de treinamento. Vamos supor que a rede geradora seja fixa e queremos otimizar o discriminador. Ambos os termos da função valor $\small V\left ( \theta ^{D}, \theta^{(G)} \right )$ contribuem para otimizar o discriminador, onde o primeiro termo corresponde à perda associada aos exemplos reais, e o segundo termo é a perda aos exemplos falsos. Portanto, quando G é fixo, nosso objetivo é *maximizar* $\small V\left ( \theta ^{D}, \theta^{(G)} \right )$, o que significa tornar o discriminador melhor na distinção entre imagens reais e geradas.


Depois de otimizar o discriminador usando os termos de perda para amostras reais e falsas, corrigimos o discriminador e otimizamos o gerador. Neste caso, apenas o segundo termo em $\small V\left ( \theta ^{D}, \theta^{(G)} \right )$ contribui para os gradientes do gerador. Como resultado, quando *D* é fixo, nosso objetivo é *minimizar* $\small V\left ( \theta ^{D}, \theta^{(G)} \right )$ , que pode ser escrito como $\small \underset{G}{min}\:E_{z\sim p_z(z))}\left [ log(1-D(G(z))) \right]$. Como foi mencionado no artigo GAN original de Goodfellow et al., esta função, $\small log(1-D(G(z)))$, sofre de gradientes de fuga nos estágios iniciais de treinamento. A razão para isso é que as saídas, $\small G_{(z)}$, no início do processo de aprendizagem, não se parecem em nada com exemplos reais e, portanto, $\small D(G_{(z)})$ será próximo de zero com alta confiança. Esse fenômeno é chamado de **saturação**. Para resolver esse problema, podemos reformular o objetivo de minimização, $\small\underset{G}{min}\:E_{z\sim p_z(z))}\left [ log(1-D(G(z))) \right ]$,
reescrevendo-o como $\small \underset{G}{max}\:E_{z\sim p_z(z))}\left [ log(D(G(z))) \right ]$.





Essa substituição significa que para treinar o gerador, podemos trocar os rótulos de exemplos reais e falsos e realizar uma minimização regular da função. Em outras palavras, mesmo que os exemplos sintetizados pelo gerador sejam falsos e, portanto, rotulados como 0, podemos inverter os rótulos atribuindo o rótulo 1 a esses exemplos e minimizar a perda de entropia cruzada binária com esses novos rótulos em vez de maximizar $\small \underset{G}{max}\:E_{z\sim p_z(z))}\left [ log(D(G(z))) \right ]$.

Agora que abordamos o procedimento geral de otimização para treinar modelos de GAN, vamos explorar os vários rótulos de dados que podemos usar ao treinar GANs.
Dado que o discriminador é um classificador binário (os rótulos de classe são 0 e 1 para imagens falsas e reais, respectivamente), podemos usar a função de perda de entropia cruzada binária (*binary cross-entropy*). Portanto, podemos determinar os rótulos de verdade do terreno para a perda do discriminador da seguinte forma:

$$
\small
\text{R$\acute{o}$tulos verdadeiros para o discriminador} = 
\begin{cases}
1:  & \quad \text{para imagens reais, por exemplo,\:$\textbf{x}$}\\ 
0: & \quad \text{para sa$\acute{i}$das de G, por exemplo, G($\textbf{z}$)}
\end{cases}
$$

E as etiquetas para treinar o gerador? Como queremos que o gerador sintetize imagens realistas, queremos penalizar o gerador quando suas saídas não forem classificadas como reais pelo discriminador. Isso significa que assumiremos os rótulos verdadeiros para as saídas do gerador como 1 ao calcular a função de perda para o gerador.

Juntando tudo isso, a figura a seguir exibe as etapas individuais em um modelo GAN simples:

<p><img src= imagens\modelo_GAN.PNG></p>

Na seção a seguir, implementaremos um GAN do zero para gerar novos dígitos manuscritos.

### Implementando um GAN do zero

Nesta seção, abordaremos como implementar e treinar um modelo GAN para gerar novas imagens, como dígitos MNIST. Como o treinamento em uma unidade de processamento central (CPU) normal **pode levar muito tempo**, na subseção a seguir, abordaremos como configurar o ambiente **Google Colab**, que nos permitirá executar os cálculos em unidades de processamento gráfico (*GPUs*) .

### Treinamento de modelos GAN no *Google Colab*
Alguns dos exemplos de código podem exigir recursos computacionais extensivos que vão além de um laptop comercial ou uma estação de trabalho sem GPU. Se você já possui uma máquina de computação habilitada para GPU NVIDIA disponível, com bibliotecas CUDA e cuDNN instaladas, você pode usá-la para acelerar os cálculos.


No entanto, como muitos de nós não têm acesso a recursos de computação de alto desempenho, usaremos o ambiente *Google Colaboratory* (muitas vezes referido como *Google Colab*), que é um serviço gratuito de computação em nuvem (disponível na maioria dos países). O *Google Colab* fornece instâncias do *Jupyter Notebook* que são executadas na nuvem; os notebooks podem ser salvos no *Google Drive* ou *GitHub*. Embora a plataforma forneça vários recursos computacionais diferentes, como CPUs, GPUs e até unidades de processamento tensor (TPUs), é importante destacar que o tempo de execução atualmente é limitado a 12 horas. Portanto, qualquer notebook executando mais de 12 horas será interrompido.

Os blocos de código precisarão de um tempo máximo de computação de duas a três horas, portanto, isso não será um problema. No entanto, se você decidir usar o *Google Colab* para outros projetos que levem mais de 12 horas, use o *checkpointing* e salve os *checkpoints* intermediários.

> ##### Notebook Jupyter
> O *Jupyter Notebook* é uma interface gráfica do usuário (GUI) para executar código de forma interativa e intercalá-lo com documentação de texto e figuras. Devido à sua versatilidade e facilidade de uso, tornou-se uma das ferramentas mais populares em ciência de dados. Para obter mais informações sobre a GUI geral do Jupyter Notebook, consulte a documentação oficial em https://jupyter-notebook.readthedocs.io/en/stable/.
> Por fim, recomendamos o artigo de Adam Rule et al. Dez regras simples para escrever e compartilhar análises computacionais em Jupyter Notebooks sobre o uso eficaz do Jupyter Notebook em projetos de pesquisa científica, disponível gratuitamente em https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007007.


O acesso ao *Google Colab* é muito simples. Você pode visitar https://colab.research.google.com, que o leva automaticamente a uma janela de prompt onde você pode ver seus notebooks Jupyter existentes. Nessa janela de prompt, clique na guia **GOOGLE DRIVE**, conforme mostrado na figura a seguir. É aqui que você salvará o bloco de anotações em seu *Google Drive*.

Em seguida, para criar um novo notebook, clique no link **NEW PYTHON 3 NOTEBOOK** na parte inferior da janela do *prompt*:

<p><img src= imagens\colab1.PNG ></p>

Isso criará e abrirá um novo bloco de anotações para você. Todos os exemplos de código que você escrever neste notebook serão salvos automaticamente e você poderá acessar o notebook posteriormente no *Google Drive* em um diretório chamado Colab Notebooks.

Na próxima etapa, queremos utilizar GPUs para executar os exemplos de código neste notebook. Para isso, na opção **Runtime** na barra de menus deste notebook, clique em **Change runtime type** e selecione **GPU**, conforme mostra a figura a seguir:
<p><img src= imagens\colab2.PNG ></p>

 
Na última etapa, só precisamos instalar os pacotes *Python* que precisaremos para realizar nossas tarefas. O ambiente *Colab Notebooks* já vem com alguns pacotes, como *NumPy*, *SciPy* e a versão estável mais recente do *TensorFlow*. No entanto, no momento da redação deste artigo, a versão estável mais recente no *Google Colab* é o TensorFlow 2.7.0. Portanto, primeiro precisamos instalar o TensorFlow 2.7 que, atualmente, **já vem com suporte a GPU**.

<code>
!pip install -q tensorflow-gpu==2.0.0
</code>

(Em um notebook Jupyter, uma célula que começa com um ponto de exclamação será interpretada como um comando de shell do Linux.)

Agora, podemos testar a instalação e verificar se a GPU está disponível usando o seguinte código:

In [4]:
import tensorflow as tf

print(f"Versão do TensorFlow: {tf.__version__}")

print(f"GPU Disponível: {tf.test.is_gpu_available()}")

Versão do TensorFlow: 2.7.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Disponível: True


Além disso, se você deseja salvar o modelo em seu *Google Drive pessoal*, ou transferir ou fazer upload de outros arquivos, é necessário montar o *Google Drive*. Para fazer isso, execute o seguinte em uma nova célula do notebook:

<code>
from google.colab import drive


drive.mount('/content/drive/')
</code>

Isso fornecerá um link para autenticar o *Colab Notebook* acessando seu *Google Drive*. Após seguir as instruções para autenticação, ele fornecerá um código de autenticação que você precisa copiar e colar no campo de entrada designado abaixo da célula que você acabou de executar. Em seguida, seu *Google Drive* será montado e estará disponível em `/content/drive/My Drive`.

### Implementação das redes geradora e discriminadora

Iniciaremos a implementação do nosso primeiro modelo GAN com um gerador e um discriminador como duas redes totalmente conectadas com uma ou mais camadas ocultas (veja a figura a seguir).

Esta é a versão original do GAN, que chamaremos de *vanilla GAN*. Neste modelo, para cada camada oculta, aplicaremos a função de ativação *ReLU* com vazamento (*Leaky ReLU*). O uso de *ReLU* resulta em gradientes esparsos, que podem não ser adequados quando queremos ter os gradientes para toda a faixa de valores de entrada. Na rede discriminadora, cada camada oculta também é seguida por uma camada *dropout*. Além disso, a camada de saída no gerador usa a função de ativação da tangente hiperbólica (*tanh*). (Usar a ativação *tanh* é recomendado para a rede do gerador, pois ajuda no aprendizado.)

A camada de saída no discriminador não possui função de ativação (ou seja, ativação linear) para obter os *logits*. Alternativamente, podemos usar a função de ativação sigmóide para obter probabilidades como saída:

<p><img src=imagens\implementacao_gan.PNG </p>

> ##### Função de ativação da unidade linear retificada com vazamento (*ReLU*)
> Anteriormente, abordamos diferentes funções de ativação não linear que podem ser usadas em um modelo de rede neural. Se você se lembra, a função de ativação *ReLU* foi definida como $\small \phi(Z) = max(0,z)$, que suprime as entradas negativas (pré-ativação); ou seja, as entradas negativas são definidas como zero. Como consequência, o uso da função de ativação *ReLU* pode resultar em gradientes esparsos durante a retropropagação. Gradientes esparsos nem sempre são prejudiciais e podem até beneficiar modelos para classificação. No entanto, em certas aplicações, como GANs, pode ser benéfico obter os gradientes para toda a faixa de valores de entrada, o que podemos conseguir fazendo uma pequena modificação na função *ReLU* de forma que ela produza pequenos valores para entradas negativas. Essa versão modificada da função *ReLU* também é conhecida como *ReLU* com vazamento (*Leaky ReLU*). Em suma, a função de ativação de *ReLU* com vazamento permite gradientes diferentes de zero também para entradas negativas e, como resultado, torna as redes mais expressivas em geral.
> A função de ativação do *ReLU* com vazamento é definida da seguinte forma:
> <p><img src= imagens\leaky_Relu.PNG></p>
>
>Aqui, $\small \alpha$ determina a inclinação para as entradas negativas (pré-ativação).

Vamos definir duas funções auxiliares para cada uma das duas redes, instanciar um modelo da classe Keras `Sequential` e adicionar as camadas conforme descrito. O código é o seguinte:

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [7]:
## Defina a Função para o Gerador:
def make_generator_network(
        num_hidden_layers=1,
        num_hidden_units=100,
        num_output_units=784):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(
            tf.keras.layers.Dense(
                units=num_hidden_units, 
                use_bias=False)
            )
        model.add(tf.keras.layers.LeakyReLU())
        
    model.add(tf.keras.layers.Dense(
        units=num_output_units, activation='tanh'))
    return model

## Defina a Função para o Discriminador:
def make_discriminator_network(
        num_hidden_layers=1,
        num_hidden_units=100,
        num_output_units=1):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(units=num_hidden_units))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(rate=0.5))
        
    model.add(
        tf.keras.layers.Dense(
            units=num_output_units, 
            activation=None)
        )
    return model

Em seguida, especificaremos as configurações de treinamento para o modelo. Como você deve se lembrar, o tamanho da imagem no conjunto de dados MNIST é de $\small 28 \times 28$ pixels. (Esse é apenas um canal de cor porque MNIST contém apenas imagens em tons de cinza.) Especificaremos ainda o tamanho do vetor de entrada, **z**, como 20, e usaremos uma distribuição uniforme aleatória para inicializar os pesos do modelo. Como estamos implementando um modelo GAN muito simples apenas para fins de ilustração e usando camadas totalmente conectadas, usaremos apenas uma única camada oculta com 100 unidades em cada rede. No código a seguir, especificaremos e inicializaremos as duas redes e imprimiremos suas informações de resumo:

In [8]:
image_size = (28, 28)
z_size = 20
mode_z = 'uniform'  # 'uniform' vs. 'normal'
gen_hidden_layers = 1
gen_hidden_size = 100
disc_hidden_layers = 1
disc_hidden_size = 100

tf.random.set_seed(1)

gen_model = make_generator_network(
    num_hidden_layers=gen_hidden_layers, 
    num_hidden_units=gen_hidden_size,
    num_output_units=np.prod(image_size))

gen_model.build(input_shape=(None, z_size))
gen_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2000      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 784)               79184     
                                                                 
Total params: 81,184
Trainable params: 81,184
Non-trainable params: 0
_________________________________________________________________


In [9]:
disc_model = make_discriminator_network(
    num_hidden_layers=disc_hidden_layers,
    num_hidden_units=disc_hidden_size)

disc_model.build(input_shape=(None, np.prod(image_size)))
disc_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 78,601
Trainable params: 78,601
Non-trainable params: 0
_________________________________________________________________


### Como definir o conjunto de dados de treinamento
Na próxima etapa, carregaremos o conjunto de dados MNIST e aplicaremos as etapas de pré-processamento necessárias. Como a camada de saída do gerador está usando a função de ativação *tanh*, os valores de pixel das imagens sintetizadas estarão no intervalo (–1, 1). No entanto, os pixels de entrada das imagens MNIST estão dentro do intervalo [0, 255] (com um tipo de dados TensorFlow `tf.uint8`). Assim, nas etapas de pré-processamento, usaremos a função `tf.image.convert_image_dtype` para converter o dtype dos tensores da imagem de entrada de `tf.uint8` para `tf.float32`. Como resultado, além de alterar o dtype, chamar essa função também alterará o intervalo de intensidades de pixel de entrada para [0, 1]. Então, podemos dimensioná-los por um fator de 2 e deslocá-los por –1, de modo que as intensidades dos pixels sejam redimensionadas para estarem no intervalo [–1, 1]. Além disso, também criaremos um vetor aleatório, **z**, com base na distribuição aleatória desejada (neste exemplo de código, uniforme ou normal, que são as escolhas mais comuns) e retornaremos a imagem pré-processada e o vetor aleatório em uma tupla:

In [11]:
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
mnist = mnist_bldr.as_dataset(shuffle_files=False)

def preprocess(ex, mode='uniform'):
    image = ex['image']
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.reshape(image, [-1])
    image = image*2 - 1.0
    if mode == 'uniform':
        input_z = tf.random.uniform(
            shape=(z_size,), minval=-1.0, maxval=1.0)
    elif mode == 'normal':
        input_z = tf.random.normal(shape=(z_size,))
    return input_z, image

mnist_trainset = mnist['train']

print('Antes do Pré-processamento:  ')
example = next(iter(mnist_trainset))['image']
print('dtype: ', example.dtype, ' Min: {} Max: {}'.format(np.min(example), np.max(example)))

mnist_trainset = mnist_trainset.map(preprocess)
print()
print('Depois do Pré-processamento:  ')
example = next(iter(mnist_trainset))[0]
print('dtype: ', example.dtype, ' Min: {} Max: {}'.format(np.min(example), np.max(example)))

Antes do Pré-processamento:  
dtype:  <dtype: 'uint8'>  Min: 0 Max: 255

Depois do Pré-processamento:  
dtype:  <dtype: 'float32'>  Min: -0.8737728595733643 Max: 0.9460210800170898


Observe que, aqui, retornamos o vetor de entrada, **z**, e a imagem para buscar os dados de treinamento convenientemente durante o ajuste do modelo. No entanto, isso não implica que o vetor, **z**, seja de alguma forma relacionado à imagem - a imagem de entrada vem do conjunto de dados, enquanto o vetor **z** é gerado aleatoriamente. Em cada iteração de treinamento, o vetor gerado aleatoriamente, **z**, representa a entrada que o gerador recebe para sintetizar uma nova imagem, e as imagens (tanto as reais quanto as sintetizadas) são as entradas do discriminador.

Vamos inspecionar o objeto de conjunto de dados que criamos. No código a seguir, pegaremos um lote de exemplos e imprimiremos as formas de matriz desta amostra de vetores e imagens de entrada. Além disso, para entender o fluxo de dados geral do nosso modelo GAN, no código a seguir, processaremos uma passagem direta para nosso gerador e discriminador.

Primeiro, vamos alimentar o lote de vetores de entrada, **z**, para o gerador e obter sua saída, `g_output`. Este será um lote de exemplos falsos, que será alimentado ao modelo discriminador para obter os logits do lote de exemplos falsos, `d_logits_fake`. Além disso, as imagens processadas que obtemos do objeto dataset serão alimentadas ao modelo discriminador, que resultará nos logits para os exemplos reais, `d_logits_real`. O código é o seguinte: